In [1]:
import os
import sys
import gzip
import code
import urllib
import importlib
import tensorflow.python.platform
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tqdm import tqdm
from PIL import Image
from datetime import datetime

In [2]:
COLAB = False
BRANCH = 'main'

In [3]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/Shareddrives/ML_Road_Segmentation/CS-433-project-ben/project_road_segmentation'
    os.chdir(drive_path)
    from helpers.colab import mount_and_pull
    BRANCH_NAME = BRANCH
    mount_and_pull(BRANCH_NAME, drive, os)

In [4]:
%load_ext autoreload
%autoreload 2

from helpers.image_processing import *
from helpers.file_manipulation import *
from helpers.constants import *
from helpers.prediction_checking import *
from helpers.machine_learning import *
from helpers.loss_functions import *
from helpers.submission import *

In [5]:
RESTORE_MODEL = False  # If True, restore existing model instead of training a new one
SAVE_MODEL = False
GENERATE_PREDICTION = False  # If True, will generate a CSV to submit on AICrowd

MODEL_NAME = 'unet_2'  # For now, cnn, unet-1, unet-2
PATCH_PREDICTION = False
SAVE_DIR = MODELS_SAVE_DIR + MODEL_NAME + '/'

NUM_EPOCHS = 250

In [6]:
mod = importlib.import_module('models.' + MODEL_NAME)
model_function = getattr(mod, MODEL_NAME)

In [17]:
FEATURE_AUGMENTATION = 2
X_train, y_train, X_test, y_test = get_train_test(data_augmentation=True,transformations=['mix','flip', 'rotation'])

Loading data/generated/rotation/groundtruth/: 100%|██████████| 100/100 [00:00<00:00, 197.35it/s]


Training features shape :  (400, 400, 400, 3)
Training labels shape :  (400, 400, 400)


In [20]:
if PATCH_PREDICTION:
    from helpers.patch_labeling import *
    old_y_train = y_train
    old_y_test = y_test
    y_train = label_to_patch_prediction(y_train)
    y_test = label_to_patch_prediction(y_test)
    print(y_train.shape)
    print(y_test.shape)
    visualize_patches(old_y_train, y_train, number=3, resize=False)

In [21]:
metrics=[
         tf.keras.metrics.Precision(),
         tf.keras.metrics.Recall(),
         dice_coef
        ]
callbacks = [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5)
            ]
if RESTORE_MODEL:
    model = tf.keras.models.load_model(SAVE_DIR, custom_objects={'focal_tversky_loss': focal_tversky_loss, 'dice_coef': dice_coef})
else:  
    model = model_function(feature_augmentation=FEATURE_AUGMENTATION+1)
    model.compile(optimizer='adam',
                loss=focal_tversky_loss,
                metrics=metrics)
    history = model.fit(X_train, y_train, 
                        epochs = NUM_EPOCHS,
                        validation_split=0.2,
                        callbacks=callbacks)

Epoch 1/250


ValueError: in user code:

    /Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Applications/anaconda3/lib/python3.7/site-packages/tensorf

In [ ]:
if SAVE_MODEL:
    now = datetime.now().strftime("%m-%d-%Y_%H-%M-%S") + '/'
    model.save(SAVE_DIR + new)

In [19]:
train_predictions = model.predict(X_train).squeeze()
test_predictions = model.predict(X_test).squeeze()

NameError: name 'model' is not defined

In [ ]:
if PATCH_PREDICTION:
    train_predictions = resize_images(train_predictions, TRAINING_IMG_SIZE)
    test_predictions = resize_images(test_predictions, TRAINING_IMG_SIZE)

In [ ]:
visualize_random_predictions(X_train, y_train, train_predictions, size=10)

In [ ]:
visualize_random_predictions(X_test, y_test, test_predictions, size=10)

In [ ]:
metrics = compute_entire_images_metrics(y_test, test_predictions)

In [ ]:
metrics

In [ ]:
#Load 608x608 test images 
images608 = load_test_images()[0]

#Split each of them into 4 400x400 images
images400 = split_608_to_400(images608)[0]

#Predict their mask
print("Predicting test images...")
masks400 = model.predict(images400).squeeze()

print(masks400.shape)
#Merge the 4 400x400 masks into one 608x608 by averaging the overlapping parts
masks608 = merge_masks400(masks400)

# binarize predictions into 0's and 1's
masks608 = np.asarray([(mask >= ROAD_THRESHOLD_PIXEL_PRED) * 1.0 for mask in masks608])

In [ ]:
predict_submissions(model)

In [ ]:
if COLAB:
    from helpers.colab import download_model
    from google.colab import files
    download_model(MODEL_NAME, SAVE_DIR, files)